<h1><center>NLP Augmentations Master Notebook</center></h1>
<h3><center>Feedback Prize - Evaluating Student Writing</center></h3>

<center><img src = "https://www.gsu.edu/wp-content/themes/gsu-flex-2/images/logo.png" width = "750" height = "500"/></center>                                                                          

After receiving great response (Gold Medal, 180+ Upvotes) for my [Image Augmentations Master Notebook](https://www.kaggle.com/ishandutta/petfinder-data-augmentations-master-notebook) I am creating a similar notebook but for Text based NLP Augmentations.

I will demonstrate with examples how you can apply a variety of transformations to your dataset based on your application!

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

> | S.No       |                   Heading                |
> | :------------- | :-------------------:                |         
> |  01 |  [**Competition Overview**](#competition-overview)  |                   
> |  02 |  [**Libraries**](#libraries)                        |  
> |  03 |  [**Global Config**](#global-config)                |
> |  04 |  [**Weights and Biases**](#weights-and-biases)      |
> |  05 |  [**Load Datasets**](#load-datasets)                |
> |  06 |  [**What is Data Augmentation?**](#what-is-data-augmentation)  |
> |  07 |  [**Basic NLP Augmentations**](#basic-nlp-augmentations)   |
> |  08 |  [**Model**](#model)

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>

---

<a id="competition-overview"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Competition Overview</center></h2>

## **<span style="color:orange;">Description</span>**


In this competition, you’ll identify elements in student writing. More specifically, you will automatically segment texts and classify argumentative and rhetorical elements in essays written by 6th-12th grade students. You'll have access to the largest dataset of student writing ever released in order to test your skills in natural language processing, a fast-growing area of data science.
  
---

## **<span style="color:orange;">Evaluation Metric</span>**

Submissions are evaluated on the overlap between ground truth and predicted word indices.

1. For each sample, all ground truths and predictions for a given class are compared.
2. If the overlap between the ground truth and prediction is >= 0.5, and the overlap between the prediction and the ground truth >= 0.5, the prediction is a match and considered a `true positive`. If multiple matches exist, the match with the highest pair of overlaps is taken.
3. Any unmatched ground truths are `false negatives` and any unmatched predictions are `false positives`.

---

<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

In [ ]:
import os
import re

import json
import time

import numpy as np
import pandas as pd

import random
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt


from nltk import sent_tokenize
from albumentations.core.transforms_interface import DualTransform, BasicTransform

from termcolor import colored

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from collections import defaultdict

import wandb
from wandb.keras import WandbCallback
wandb.login()

---

<a id="global-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Global Config</center></h2>

In [ ]:
class config:
    SEED = 42
    DIRECTORY_PATH = "../input/feedback-prize-2021"
    TRAIN_CSV_PATH = os.path.join(DIRECTORY_PATH, 'train.csv')

In [ ]:
def set_seed(seed=config.SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
set_seed()

---

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases</center></h2>

<center><img src = "https://i.imgur.com/1sm6x8P.png" width = "750" height = "500"/></center>  

**Weights & Biases** is the machine learning platform for developers to build better models faster.

You can use W&B's lightweight, interoperable tools to

- quickly track experiments,
- version and iterate on datasets,
- evaluate model performance,
- reproduce models,
- visualize results and spot regressions,
- and share findings with colleagues.
  
Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In this notebook I will use Weights and Biases's amazing features to perform wonderful visualizations and logging seamlessly.

---

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

In [ ]:
train = pd.read_csv(config.TRAIN_CSV_PATH)

In [ ]:
train.head()

In [ ]:
submission = pd.read_csv("../input/feedback-prize-2021/sample_submission.csv")

In [ ]:
submission.head()

<a id="what-is-data-augmentation"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>What is Data Augmentation</center></h2>

## **<span style="color:orange;">Definition</span>**

Data Augmentation is a technique used to increase the amount of data by adding slightly modified copies of already existing data or newly created synthetic data from existing data.

## **<span style="color:orange;">Why is Data Augmentation Important?</span>**
Data augmentation is useful to improve performance and outcomes of machine learning models by forming new and different examples to train datasets. If dataset in a machine learning model is rich and sufficient, the model performs better and more accurate.
  
For machine learning models, collecting and labeling of data can be exhausting and costly processes. Transformations in datasets by using data augmentation techniques allow companies to reduce these operational costs.

---

<a id="basic-nlp-augmentations"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Basic NLP Augmentations</center></h2>

In this section we will look at some simple transforms that can be performed on Text Data. To develop this section I am referring to [this](https://www.kaggle.com/shonenkov/nlp-albumentations) notebook by Alex Shonenkov. I will build upon those transforms with more examples and techniques.

**I will demonstrate the 7 basic transforms, namely:**
1. [Sentence Shuffling](#basic-1)
2. [Remove Duplicate Sentences](#basic-2)
3. [Remove Numbers](#basic-3)
4. [Remove Hashtags](#basic-4)
5. [Remove Mentions](#basic-5)
6. [Remove URLs](#basic-6)
7. [Cut Out Words](#basic-7)

In [ ]:
class TextAugmentation(BasicTransform):
    
    """ 
    Transform for NLP task.
    """

    @property
    def targets(self):
        return {"data": self.apply}
    
    def update_params(self, params, **kwargs):
        if hasattr(self, "interpolation"):
            params["interpolation"] = self.interpolation
        if hasattr(self, "fill_value"):
            params["fill_value"] = self.fill_value
        return params

    def get_sentences(self, text):
        return sent_tokenize(text)

In [ ]:
def demo(augmentation, text):
    
    """
    Function to demonstrate the applied augmentation.
    
    params:
        augmentation - The augmentation to be applied on the text
        text: Text on which transform will be applied
        
    """
    
    output = augmentation(data=(text))['data']

    print("Original Text")
    print(colored(text, 'red'))
    
    print()
    
    print("Augmented Text")
    print(colored(output, 'yellow'))

---

## **<span style="color:orange;">1. Sentence Shuffling</span>**
<a id="basic-1"></a>

In Sentence Shuffling we will randomly shuffle the sentences of the text.

In [ ]:
class SentenceShuffleAugmentation(TextAugmentation):
    """ Shuffle the sentences of the text. """
    
    def __init__(self, always_apply=False, p=0.5):
        super(SentenceShuffleAugmentation, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text = data
        sentences = self.get_sentences(text)
        random.shuffle(sentences)
        return ' '.join(sentences)

In [ ]:
demo(
    augmentation = SentenceShuffleAugmentation(p=1.0),
    text = train.iloc[0, 4]
)

## **<span style="color:orange;">2. Remove Duplicate Sentences</span>**
<a id="basic-2"></a>

In Remove Duplicates we will remove duplicate sentences from the text. 

To demonstrate this, we concatenate a sentence with itself and output should only be the original text.

In [ ]:
class RemoveDuplicateSentencesAugmentation(TextAugmentation):
    """ Exclude equal sentences """
    def __init__(self, always_apply=False, p=0.5):
        super(RemoveDuplicateSentencesAugmentation, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text = data
        sentences = []
        for sentence in self.get_sentences(text):
            sentence = sentence.strip()
            if sentence not in sentences:
                sentences.append(sentence)
        return ' '.join(sentences)

In [ ]:
demo(
    augmentation = RemoveDuplicateSentencesAugmentation(p=1.0),
    text = train.iloc[0, 4][0:46] + " " + train.iloc[0, 4][0:46]
)

---

## **<span style="color:orange;">3. Remove Numbers</span>**
<a id="basic-3"></a>

In Remove Numbers, we will remove any number from the text. We can simply achieve this using regular expressions.

In [ ]:
class RemoveNumbersAugmentation(TextAugmentation):
    """ Exclude any numbers """
    def __init__(self, always_apply=False, p=0.5):
        super(RemoveNumbersAugmentation, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text = data
        text = re.sub(r'[0-9]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text

In [ ]:
demo(
    augmentation = RemoveNumbersAugmentation(p=1.0),
    text = "There are 15594 samples of training data."
)

---

## **<span style="color:orange;">4. Remove Hashtags</span>**
<a id="basic-4"></a>

In Remove Hashtags, we will remove any hashtag from the text. 

In [ ]:
class RemoveHashtagsAugmentation(TextAugmentation):
    """ Exclude any hashtags with # """
    def __init__(self, always_apply=False, p=0.5):
        super(RemoveHashtagsAugmentation, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text = data
        text = re.sub(r'#[\S]+\b', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text

In [ ]:
demo(
    augmentation = RemoveHashtagsAugmentation(p=1.0),
    text = "Kaggle Competitions are fun. #MachineLearning"
)

---

## **<span style="color:orange;">5. Remove Mentions</span>**
<a id="basic-5"></a>

In this transform we remove any mentions (word beginning with '@') from the text.

In [ ]:
class RemoveMentionsAugmentation(TextAugmentation):
    """ Exclude @users """
    
    def __init__(self, always_apply=False, p=0.5):
        super(RemoveMentionsAugmentation, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text = data
        text = re.sub(r'@[\S]+\b', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text

In [ ]:
demo(
    augmentation = RemoveMentionsAugmentation(p=1.0),
    text = "@AnthonyGoldbloom is the founder of Kaggle."
)

---

## **<span style="color:orange;">6. Remove URLs</span>**
<a id="basic-6"></a>

In this transform we remove any URLs from the text.

In [ ]:
class RemoveUrlAugmentation(TextAugmentation):
    """ Exclude urls """
    
    def __init__(self, always_apply=False, p=0.5):
        super(RemoveUrlAugmentation, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text = data
        text = re.sub(r'https?\S+', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text

In [ ]:
demo(
    augmentation = RemoveUrlAugmentation(p=1.0),
    text = "https://www.kaggle.com hosts the world's best Machine Learning Hackathons."
)

---

## **<span style="color:orange;">7. Cut Out Words</span>**
<a id="basic-7"></a>

In this transform, we remove some words from the text.

In [ ]:
class CutOutWordsAugmentation(TextAugmentation):
    """ Remove random words """
    
    def __init__(self, cutout_probability=0.05, always_apply=False, p=0.5):
        super(CutOutWordsTransform, self).__init__(always_apply, p)
        self.cutout_probability = cutout_probability

    def apply(self, data, **params):
        text = data
        words = text.split()
        words_count = len(words)
        if words_count <= 1:
            return text
        
        new_words = []
        for i in range(words_count):
            if random.random() < self.cutout_probability:
                continue
            new_words.append(words[i])

        if len(new_words) == 0:
            return words[random.randint(0, words_count-1)]

        return ' '.join(new_words)

In [ ]:
demo(
    augmentation = CutOutWordsTransform(p=1.0, cutout_probability=0.2),
    text = "Competition objective is to analyze argumentative writing elements from students grade 6-12."
)

<a id="model"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Model</center></h2>

I am demonstrating a simple NER model where you can try applying above transforms and see the change in results. 

Note that some of these transforms might not be suitable entirely for NER, you can try them for other applications as well. For the model I am referring to [this](https://www.kaggle.com/lonnieqin/name-entity-recognition-with-keras) notebook.

## **<span style="color:orange;">Model Configuration</span>**

In [ ]:
vocab_size = 10000                    # Vocabulary size
sequence_length = 1024                # Sequence Length
batch_size = 128                      # Batch size
unk_token = "<unk>"                   # Unknownd token
vectorizer_path = "vectorizer.json"

# Use output dataset for inference
output_dataset_path = "../input/name-entity-recognition-with-keras-output/"
model_path = "model.h5"
embed_size = 64
hidden_size = 64

modes = ["training", "inference"]     # There is training and inference mode
mode = modes[1]
epochs = 10
dropout = 0.2                         # Dropout rate for the Model

In [ ]:
# wandb config
WANDB_CONFIG = {
     'competition': 'Feedback Prize', 
              '_wandb_kernel': 'neuracort'
    }

## **<span style="color:orange;">Preprocessing</span>**

In [ ]:
train["file_path"] = train["id"].apply(
    lambda item: "../input/feedback-prize-2021/train/" + item + ".txt"
)

submission["file_path"] = submission["id"].apply(
    lambda item: "../input/feedback-prize-2021/test/" + item + ".txt"
)

discourse_types = np.array(
    ["<PAD>", "<None>"] + sorted(train["discourse_type"].unique())
)

discourse_types_index = dict(
    [(discoure_type, index) for (index, discoure_type) in enumerate(discourse_types)]
)

## **<span style="color:orange;">Tokenization</span>**

In [ ]:
def get_range(item):
    locations = [int(location) for location in item["predictionstring"].split(" ")]
    return (locations[0], locations[-1])

In [ ]:
character_counter = defaultdict(int)
character_counter
allow_set = set("'&%-_/$+ÂÃÅËÓâåóþ@|~¢£¢£")

def tokenize(text):
    
    tokens = []
    chars = []
    
    for i in range(len(text)):
        c = text[i].lower()
        character_counter[c] += 1
        is_valid = c.isalnum() or c in allow_set
    
        if i >= 1 and i < len(text) - 1:
            if text[i-1].isdigit() and text[i+1].isdigit():
                is_valid = True
            elif text[i-1].isalpha() and text[i+1].isalpha() and c == ".":
                is_valid = True
        
        if is_valid:
            chars.append(c)
        
        if (not is_valid or i == len(text) - 1) and len(chars) > 0:
            tokens.append("".join(chars))
            chars.clear()
    
    return tokens

In [ ]:
%%time

begin = time.time()
last_id = ""
contents = []
wrong_samples = []
token_list = []
annotation_list = []

num_samples = len(train)
unmaptch_count = 0              # Number of sentences extracted from predictionstring that doesn't discourse_text
match_count = 0                 # Number of sentences extracted from predictionstring that matches discourse_text including shifting
completely_match_count = 0      # Number of sentences extracted from predictionstring that matches discourse_text without shifting
mismatch_count = 0

for i in range(len(train)):
    item = train.iloc[i]
    identifier = item["id"] 
    discourse_type_id = discourse_types_index[item["discourse_type"]]
    
    if identifier != last_id:
        last_id = identifier
    
        with open(item["file_path"]) as f:
            content = "".join(f.readlines())
            contents.append(content)
            tokens = tokenize(content)
            token_list.append(tokens)
            annotations = [1] * len(tokens)
            annotation_list.append(annotations)
    
    annotation_range = get_range(item)
    extracted = tokens[annotation_range[0]:annotation_range[1]+1]
    discourse = tokenize(item["discourse_text"])
    delta = None
    num_tokens_to_compare = min(len(discourse), 3)
    
    # Compare text extracted from predictionstring with discourse_text, shift discourse_text or right if needed, just compare a few words for performance
    for j in range(10):
    
        if len(extracted) < num_tokens_to_compare or len(discourse) <= j + num_tokens_to_compare:
            break
        
        if extracted[0:num_tokens_to_compare] == discourse[j:num_tokens_to_compare+j]:
            delta = j
            break
    
    if delta == None:
        for j in range(10):
            if len(discourse) < num_tokens_to_compare and len(extracted) <= j + num_tokens_to_compare:
                break
            
            if discourse[0:num_tokens_to_compare] == extracted[j:num_tokens_to_compare+j]:
                delta = -j
                break
    
    if delta == None:
        unmaptch_count += 1
    
    else:
        not_match = False
        for j in range(annotation_range[0] - delta, min(min(annotation_range[1] - delta + 1, len(tokens)), len(discourse) + annotation_range[0] - delta)): 
            if tokens[j] != discourse[j - annotation_range[0] + delta]:
                mismatch_count += 1
                not_match = True
                break
        
        if not not_match:
            for j in range(annotation_range[0] - delta, min(min(annotation_range[1] - delta + 1, len(tokens)), len(discourse) + annotation_range[0] - delta)): 
                annotation_list[-1][j] = discourse_type_id
            match_count += 1
        
        else:
            unmaptch_count += 1
        
        if delta == 0:
            completely_match_count += 1 

print("Unmatch count:%d Match Count: %d Completedly Match count: %d"%(unmaptch_count, match_count, completely_match_count))
print("Mismatch count:", mismatch_count)
print(token_list[0])
print(annotation_list[0])

## **<span style="color:orange;">Vectorization</span>**

In [ ]:
class Vectorizer:
    
    def __init__(self, vocab_size = None, sequence_length = None, unk_token = "<unk>"):
        
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.unk_token = unk_token
        
    def fit_transform(self, sentences):
        
        word_counter = dict()
        
        for tokens in sentences:
            for token in tokens: 
                if token in word_counter:
                    word_counter[token] += 1
                else:
                    word_counter[token] = 1
        
        word_counter = pd.DataFrame({"key": word_counter.keys(), "count": word_counter.values()})
        word_counter.sort_values(by="count", ascending=False, inplace=True)
        vocab = set(word_counter["key"][0:self.vocab_size-1])
        
        word_index = dict()
        begin_index = 1 
        word_index[self.unk_token] = begin_index
        begin_index += 1
        
        Xs = []
        
        for i in range(len(sentences)):
            X = []
            
            for token in sentences[i]:
                
                if token not in word_index and token in vocab:
                    word_index[token] = begin_index
                    begin_index += 1
                
                if token in word_index:
                    X.append(word_index[token])
                
                else:
                    X.append(word_index[self.unk_token])
                
                if len(X) == self.sequence_length:
                    break
            
            if len(X) < self.sequence_length:
                X += [0] * (self.sequence_length - len(X))
            
            Xs.append(X)
        
        self.word_index = word_index
        self.vocab = vocab
        
        return Xs
    
    def transform(self, sentences):
        
        Xs = []
        
        for i in range(len(sentences)):
            X = []
            
            for token in sentences[i]:
                if token in self.word_index:
                    X.append(self.word_index[token])
                else:
                    X.append(self.word_index[self.unk_token])
                if len(X) == self.sequence_length:
                    break
            
            if len(X) < self.sequence_length:
                X += [0] * (self.sequence_length - len(X))
            Xs.append(X)
        
        return Xs
    
    def load(self, path):
        
        with open(path, 'r') as f:
            
            dic = json.load(f)
            self.vocab_size = dic['vocab_size']
            self.sequence_length = dic['sequence_length']
            self.unk_token = dic['unk_token']
            self.word_index = dic['word_index']
            
    def save(self, path):
        
        with open(path, 'w') as f:
            
            data = json.dumps({
                "vocab_size": self.vocab_size, 
                "sequence_length": self.sequence_length, 
                "unk_token": self.unk_token,
                "word_index": self.word_index
            })
            
            f.write(data)

In [ ]:
%%time

vectorizer = Vectorizer(vocab_size = vocab_size, sequence_length = sequence_length, unk_token = unk_token)

if mode == modes[0]:
    Xs = vectorizer.fit_transform(token_list)
    vectorizer.save(vectorizer_path)

else:
    vectorizer.load(output_dataset_path + vectorizer_path)
    Xs = vectorizer.transform(token_list)

ys = []
annotation_count = [0] * len(discourse_types_index)

for annotation in annotation_list:
    if len(annotation) <= sequence_length:
        ys.append(annotation + [0] * (sequence_length - len(annotation)))
    else:
        ys.append(annotation[0:sequence_length])
    for item in ys[-1]:
        annotation_count[item] += 1

X_train, X_val, y_train, y_val = train_test_split(np.array(Xs), np.array(ys), test_size = 0.2, random_state=42)

## **<span style="color:orange;">Dataset</span>**

In [ ]:
def make_dataset(X, y, batch_size, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if mode == "train":
        ds = ds.shuffle(512)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    
    return ds

In [ ]:
train_ds = make_dataset(X_train, y_train, batch_size)
val_ds = make_dataset(X_val, y_val, batch_size, mode="valid")

## **<span style="color:orange;">Model</span>**

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embed_size, input_length=sequence_length),
    keras.layers.SpatialDropout1D(dropout),
    keras.layers.Bidirectional(keras.layers.LSTM(hidden_size, dropout=dropout, recurrent_dropout=dropout)),
    keras.layers.RepeatVector(sequence_length),
    keras.layers.Bidirectional(keras.layers.LSTM(hidden_size, return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(len(discourse_types), activation="softmax"))
])

model.summary()

In [ ]:
# Initialize W&B
run = wandb.init(project='feedback-prize-nlp-augmentations', config=WANDB_CONFIG)

if mode == modes[0]:
    checkpoint = keras.callbacks.ModelCheckpoint(
        model_path, 
        save_best_only=True,
        save_weights_only=True
    )
    
    early_stop = keras.callbacks.EarlyStopping(
        min_delta=1e-4, 
        patience=10
    )
    
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        patience=2, 
        min_lr=1e-7
    )
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    callbacks = [early_stop, checkpoint, reduce_lr, WandbCallback()]
    optimizer = tf.keras.optimizers.Adam(1e-3)
    model.compile(loss=loss, optimizer=optimizer)
    model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)
    wandb.log({'loss':loss})

else:
    model.load_weights(output_dataset_path + model_path)
    
wandb.finish()

--- 

## **<span style="color:orange;">Let's have a Talk!</span>**
> ### Reach out to me on [LinkedIn](https://www.linkedin.com/in/ishandutta0098)

---